In [1]:
import pandas as pd 
import numpy as np 
Ndata = pd.read_csv('E:/blackblaze/NegativeSample.csv')D

In [2]:
Pdata = pd.read_csv('E:/blackblaze/positiveSample.csv')

In [3]:
data = Ndata.sort_values(by = 'serial_number')

In [7]:
data = data.iloc[0:50000, :]

In [5]:
data = data.sort_values(by = ['date', 'serial_number'])

In [8]:
data[data.serial_number == 'S3000A9T']

,Unnamed: 0,capacity_bytes,date,failure,model,serial_number,smart_190_raw,smart_193_raw,smart_194_raw,smart_1_raw,smart_241_raw,smart_7_raw
0,0,4000787030016,2018-01-01,0,ST4000DM000,S3000A9T,22.0,64132.0,22.0,38941024.0,1.106223e+10,173472387.0
31931,31931,4000787030016,2018-01-02,0,ST4000DM000,S3000A9T,23.0,64132.0,23.0,204125824.0,1.107356e+10,174237590.0


In [9]:
class Mydata(Dataset):
    def __init__(self, PositiveSample, NegativeSample, numstep):
        self.serial_numberOfPositive = list(PositiveSample['serial_number'].value_counts().index)
        self.serial_numberOfNegative = list(NegativeSample['serial_number'].value_counts().index)

        self.PositiveSample_fea = PositiveSample.iloc[:, 6:12]
        self.NegativeSample_fea = NegativeSample.iloc[:, 6:12]

        self.colOfPositive = self.PositiveSample_fea.columns
        self.colOfNegative = self.PositiveSample_fea.columns

        self.fea_num = 6

        self.X_som = []
        self.y_smo = []
        '''

        for i in self.colOfPositive:
            max = np.max(PositiveSample[i])
            min = np.min(PositiveSample[i])
            scalar = max - min 
            PositiveSample[i] = PositiveSample[i] / scalar
        
        for i in self.colOfNegative:
            max = np.max(NegativeSample[i])
            min = np.min(NegativeSample[i])
            scalar = max - min 
            NegativeSample[i] = NegativeSample[i] / scalar
        '''

        for i in self.colOfPositive:
            mean = np.mean(PositiveSample[i])
            std = np.std(PositiveSample[i])
            PositiveSample[i] = (PositiveSample[i] - mean) / std


        for i in self.colOfNegative:
            mean = np.mean(NegativeSample[i]
            std = np.std(NegativeSample[i])
            NegativeSample[i] = (NegativeSample[i] - mean) / std

        self.inputs = []
        self.labels = []
        
        self.inputs_som = []
        self.labels_som = []


        for i in self.serial_numberOfPositive:
            Positive_bySerial = PositiveSample[PositiveSample['serial_number'] == i].iloc[:, 6: 12].values
            for j in range(len(Positive_bySerial) - numstep):
                self.inputs.append(torch.Tensor(np.array(Positive_bySerial[j: j + numstep])))
                self.labels.append(torch.ones(1))

        for i in self.serial_numberOfNegative:
            Negetive_bySerial = NegativeSample[NegativeSample['serial_number'] == i].iloc[:, 6: 12].values
            for j in range(len(Negetive_bySerial) - numstep):
                self.inputs.append(torch.Tensor(np.array(Negetive_bySerial[j: j + numstep])))
                self.labels.append(torch.zeros(1))
        
        for i in range(len(self.inputs)):
            self.inputs[i] = np.array(self.inputs[i].reshape(numstep * self.fea_num))
        
        self.X_som, self.y_smo = SMOTE(random_state=42).fit_sample(self.inputs, self.labels)

        for i in self.X_som:
            i = torch.Tensor(i)
            self.inputs_som.append(i.reshape(5, 6))

        for i in self.y_smo:
            if(i == 1):
                self.labels_som.append(torch.ones(1))
            else:
                self.labels_som.append(torch.zeros(1))

    def __len__(self):
        return len(self.inputs_som)

    def __getitem__(self, idx):
        return self.inputs_som[idx], self.labels_som[idx]

    def get(self):
        return self.inputs_som, self.labels_som

In [10]:
PositiveSample = Pdata
NegativeSample = data



In [11]:
train_set = Mydata(PositiveSample, NegativeSample, 5)

In [25]:
inputs, labels = train_set.get()

In [12]:
inputs2 = inputs
labels2 = labels

In [17]:
for i in range(len(inputs2)):
    inputs2[i] = np.array(inputs2[i].reshape(30))

In [18]:
smo = SMOTE(random_state=42)
X_smo, y_smo = smo.fit_sample(inputs2, labels2)

In [20]:
Counter(y_smo)

Counter({1.0: 48939, 0.0: 48939})

In [32]:
inputs3 = []
labels3 = []
for i in X_smo:
    i = torch.Tensor(i)
    inputs3.append(i.reshape(5, 6))
for i in y_smo:
    labels3.append(torch.Tensor(int(i)))

In [28]:
type(inputs[0])

torch.Tensor

In [13]:
train_iter = DataLoader(train_set, batch_size = 64, shuffle = True)

In [89]:
i = 0
for X, y in train_iter:
    print(X)
    print(y.shape)
    break

tensor([[[ 0.3372, -0.6699,  0.3372,  1.6957, -1.4575, -0.0253],
         [ 0.2162, -0.6699,  0.2162, -0.6394, -1.4580, -0.0253],
         [ 0.2162, -0.6699,  0.2162,  1.2561, -1.4575, -0.0253],
         [ 0.2162, -0.6699,  0.2162,  0.1570, -1.4573, -0.0253],
         [ 0.1898, -0.6699,  0.1898,  0.4434, -1.4567, -0.0253]],

        [[-0.6071, -0.4330, -0.6071,  0.1462, -0.6541, -0.0252],
         [-0.4939, -0.4329, -0.4939,  0.5924, -0.6539, -0.0252],
         [-0.6071, -0.4330, -0.6071, -1.0112, -0.6533, -0.0252],
         [-0.5729, -0.4330, -0.5729,  0.7767, -0.6537, -0.0252],
         [-0.5387, -0.4330, -0.5387,  0.2340, -0.6536, -0.0252]],

        [[ 1.6478, -0.5148,  1.6478, -0.1483,  1.1076, -0.0253],
         [ 1.5594, -0.5148,  1.5594,  1.5332,  1.1093, -0.0253],
         [ 1.6775, -0.5148,  1.6775,  0.5425,  1.1106, -0.0253],
         [ 1.5594, -0.5148,  1.5594, -0.0772,  1.1119, -0.0253],
         [ 1.5594, -0.5147,  1.5594, -0.3794,  1.1131, -0.0253]],

        ...,

     

In [74]:
len(labels)

97878

In [35]:
len(inputs)

97878

In [38]:
inputs[1]

tensor([[ 0.7637, -0.6067,  0.7637,  0.6392,  0.2936, -0.0252],
        [ 0.7637, -0.6067,  0.7637,  1.3676,  0.2924, -0.0252],
        [ 0.6164, -0.6067,  0.6164, -1.1836,  0.2910, -0.0252],
        [ 0.7637, -0.6067,  0.7637,  1.3528,  0.2898, -0.0252],
        [ 0.6164, -0.6067,  0.6164, -1.1616,  0.2878, -0.0252]])

In [10]:
import pandas as pd 
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
from imblearn.over_sampling import SMOTE
from collections import Counter

device = torch.device('cpu')
print(device)

class diskLSTM(nn.Module):
    def __init__(self):
        super(diskLSTM, self).__init__()

        self.LSTM_1 = nn.LSTM(input_size=6, hidden_size=32, batch_first=True)
        self.LSTM_2 = nn.LSTM(input_size=32, hidden_size=64, batch_first=True)
        self.LSTM_3 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
 
        self.linear_1 = nn.Linear(128, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 1)
    
    def forward(self, X, state):
        X, _ = self.LSTM_1(X, state)
        X, _ = self.LSTM_2(X, state)
        X, _ = self.LSTM_3(X, state)

        X = X[:, X.size(1) - 1, :]


        X = self.linear_1(X)

        X = F.tanh(X)
        X = self.linear_2(X)

        X = F.tanh(X)
        X = self.linear_3(X)

        X = torch.sigmoid(X)

        return X

class Mydata(Dataset):
    def __init__(self, PositiveSample, NegativeSample, numstep):
        self.serial_numberOfPositive = list(PositiveSample['serial_number'].value_counts().index)
        self.serial_numberOfNegative = list(NegativeSample['serial_number'].value_counts().index)

        self.PositiveSample_fea = PositiveSample.iloc[:, 6:12]
        self.NegativeSample_fea = NegativeSample.iloc[:, 6:12]

        self.colOfPositive = self.PositiveSample_fea.columns
        self.colOfNegative = self.PositiveSample_fea.columns

        self.fea_num = 6

        self.X_som = []
        self.y_smo = []
        '''

        for i in self.colOfPositive:
            max = np.max(PositiveSample[i])
            min = np.min(PositiveSample[i])
            scalar = max - min 
            PositiveSample[i] = PositiveSample[i] / scalar
        
        for i in self.colOfNegative:
            max = np.max(NegativeSample[i])
            min = np.min(NegativeSample[i])
            scalar = max - min 
            NegativeSample[i] = NegativeSample[i] / scalar
        '''

        for i in self.colOfPositive:
            mean = np.mean(PositiveSample[i])
            std = np.std(PositiveSample[i])
            PositiveSample[i] = (PositiveSample[i] - mean) / std


        for i in self.colOfNegative:
            mean = np.mean(NegativeSample[i])
            std = np.std(NegativeSample[i])
            NegativeSample[i] = (NegativeSample[i] - mean) / std

        self.inputs = []
        self.labels = []
        
        self.inputs_som = []
        self.labels_som = []


        for i in self.serial_numberOfPositive:
            Positive_bySerial = PositiveSample[PositiveSample['serial_number'] == i].iloc[:, 6: 12].values
            for j in range(len(Positive_bySerial) - numstep):
                self.inputs.append(torch.Tensor(np.array(Positive_bySerial[j: j + numstep])))
                self.labels.append(torch.ones(1))

        for i in self.serial_numberOfNegative:
            Negetive_bySerial = NegativeSample[NegativeSample['serial_number'] == i].iloc[:, 6: 12].values
            for j in range(len(Negetive_bySerial) - numstep):
                self.inputs.append(torch.Tensor(np.array(Negetive_bySerial[j: j + numstep])))
                self.labels.append(torch.zeros(1))
        
        for i in range(len(self.inputs)):
            self.inputs[i] = np.array(self.inputs[i].reshape(numstep * self.fea_num))
        
        self.X_som, self.y_smo = SMOTE(random_state=42).fit_sample(self.inputs, self.labels)

        for i in self.X_som:
            i = torch.Tensor(i)
            self.inputs_som.append(i.reshape(5, 6))

        for i in self.y_smo:
            if(i == 1):
                self.labels_som.append(torch.ones(1))
            else:
                self.labels_som.append(torch.zeros(1))

    def __len__(self):
        return len(self.inputs_som)

    def __getitem__(self, idx):
        return self.inputs_som[idx], self.labels_som[idx]

    def get(self):
        return self.inputs_som, self.labels_som

def classifyRes(y_hat):
    for i in range(y_hat.size(0)):
        if(y_hat[i] > 0.5):
            y_hat[i] = 1
        else:
            y_hat[i] = 0
    return y_hat


def train(net, train_iter, optimizer, num_epochs):
    print("training start")
    loss = torch.nn.BCELoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        state = None
        for X, y in train_iter:
            X = X.to(device)
            y = y.squeeze(1)
            y = y.to(device)
            y_hat = net(X.float(), state)
            y_hat = y_hat.to(device)
            l = loss(y_hat, y.float())
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            y_hat = classifyRes(y_hat)
            train_acc_sum += (y_hat == y).sum().item()
            n += y.size(0)
            batch_count += 1
        print('epoch %d, loss %.4f, train acc %.3f, time %.1f sec'
        % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, time.time() - start))





PositiveSample = Pdata
NegativeSample = data

train_set = Mydata(PositiveSample, NegativeSample, 5)
train_iter = DataLoader(train_set, batch_size = 64, shuffle = True)


net = diskLSTM()
net = net.to(device)
lr, num_epochs = 0.01, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

train(net, train_iter, optimizer, num_epochs)





cpu
training start
epoch 1, loss 0.0437, train acc 32.479, time 18.2 sec
epoch 2, loss 0.0398, train acc 32.516, time 17.6 sec
epoch 3, loss 0.0091, train acc 32.490, time 17.4 sec
epoch 4, loss 0.0046, train acc 32.479, time 17.8 sec
epoch 5, loss 0.0111, train acc 32.531, time 17.7 sec
epoch 6, loss 0.0274, train acc 32.494, time 17.9 sec
epoch 7, loss 0.0140, train acc 32.467, time 18.1 sec
epoch 8, loss 0.0115, train acc 32.462, time 17.9 sec
epoch 9, loss 0.0186, train acc 32.513, time 18.3 sec
epoch 10, loss 0.0280, train acc 32.516, time 18.2 sec


In [15]:
smo = SMOTE(random_state=42)
a = []
b = []
a.append(np.arange(30))
a.append(np.arange(30))
b.append(0)
b.append(1)
A, B = smo.fit_sample(a, b)

In [16]:
a[0].shape

(30,)

In [39]:
x, y = train_set.get()

In [40]:
len(y)

438

In [42]:
sum(y)

tensor([438.])

In [21]:
torch.cuda.get_device_name(0)

&#39;Quadro P400&#39;

In [22]:
torch.cuda.current_device()

0

In [23]:
torch.cuda.device_count()

1

In [24]:
torch.cuda.is_available()

True

In [27]:
a = 

device(type=&#39;cuda&#39;)

In [35]:
x = torch.ones(64, 2)

In [36]:
x = F.softmax(x)

In [39]:
x.size()

torch.Size([64, 2])

In [41]:
for a in x:
    print(a)
    break

tensor([0.5000, 0.5000])


In [3]:
import json
import torch

In [4]:
l = []
l.append(torch.ones(1))
a = json.dumps(l)

TypeError: Object of type Tensor is not JSON serializable

In [5]:
 obj={'mat':torch.randn(10, 10),'name': '10','test':{'entry':1}}

In [8]:
torch.save(l, 'E:/blackblaze/a.dat')

In [9]:
b = torch.load('E:/blackblaze/a.dat')

In [13]:
type(b)

list

In [27]:
train_set.__getitem__

ValueError: too many values to unpack (expected 2)

In [1]:
from imblearn.over_sampling import SMOTE

In [18]:
from sklearn.datasets import make_classification

In [19]:
from collections import Counter

In [20]:

X, y = make_classification(n_classes=2, class_sep=2,
                           weights=[0.9, 0.1], n_informative=3, 
                           n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1, 
                           n_samples=1000, random_state=10)

In [20]:
X[0]

array([ 0.60013068, -1.42766402, -0.8128431 ,  0.18619638, -0.44607171,
        1.46153788, -0.20792618, -0.12137372, -0.06446773, -1.33101647,
       -1.66919399, -0.13944961,  0.03095361, -0.51795082, -0.42962235,
       -0.45009542,  2.30385329, -0.55302378,  0.14876985, -1.74795877])

In [21]:
l = []
for i in y:
    l.append(i)
a = []
for i in X:
    a.append(i)

In [29]:
a[0].shape

(20,)

In [22]:
print(Counter(y))

Counter({0: 900, 1: 100})


In [8]:
type(y)

numpy.ndarray

In [23]:
smo = SMOTE(random_state=42)
X_smo, y_smo = smo.fit_sample(a, l)

In [35]:
type(X_smo[0])

list

In [ ]:
Counter(y_smo)

In [12]:
smo = SMOTE(ratio={1: 300},random_state=42)
X_smo, y_smo = smo.fit_sample(X, y)
print(Counter(y_smo))

TypeError: __init__() got an unexpected keyword argument &#39;ratio&#39;

In [14]:
from imblearn.under_sampling import RandomUnderSampler
# 同理，也可使用ratio来指定下采样的比例
rus = RandomUnderSampler(random_state=0)
X_rus, y_rus = rus.fit_sample(X, y)
print(Counter(y_smo))
# Counter({0: 500, 1: 300})

Counter({0: 900, 1: 900})


In [ ]:
from imblearn.over_sampling import ADASYN
X_adam y_ada = ADASYN().fit_sample(X, y)
print(Counter())

In [36]:
train_set = Mydata(PositiveSample, NegativeSample, 5)

(5, 6)


ValueError: Found array with dim 3. Estimator expected &lt;= 2.